In [3]:
from pyspark.sql import functions as F

csv_path = r"D:\Columbia\Fall2025\5400\project\All_external.csv"

df = (
    spark.read
    .format("csv")
    .options(
        header='true',
        inferSchema='true',
        treatEmptyValuesAsNulls='true'
    )
    .load(csv_path)
)

df.printSchema()
print("Count of all records:", df.count())
print(df.columns)


root
 |-- Date: string (nullable = true)
 |-- Article_title: string (nullable = true)
 |-- Stock_symbol: string (nullable = true)
 |-- Url: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- Article: string (nullable = true)
 |-- Lsa_summary: string (nullable = true)
 |-- Luhn_summary: string (nullable = true)
 |-- Textrank_summary: string (nullable = true)
 |-- Lexrank_summary: string (nullable = true)

Count of all records: 29984720
['Date', 'Article_title', 'Stock_symbol', 'Url', 'Publisher', 'Author', 'Article', 'Lsa_summary', 'Luhn_summary', 'Textrank_summary', 'Lexrank_summary']


### Bronze Layer

In [1]:
from pyspark.sql import SparkSession, Row
import os

# 初始化 SparkSession
spark = SparkSession.builder \
    .appName("TestParquet") \
    .getOrCreate()

print("Spark alive?", spark.sparkContext._jsc.sc().isStopped() == False)

test_df = spark.createDataFrame([
    Row(id=1, txt="hello"),
    Row(id=2, txt="world")
])

test_path = r"D:\Columbia\Fall2025\5400\project\test_parquet"

# 如果文件夹存在先删除
if os.path.exists(test_path):
    import shutil
    shutil.rmtree(test_path)

(
    test_df
    .write
    .mode("overwrite")
    .parquet(test_path)
)

print("Write OK:", os.path.exists(test_path))

spark.read.parquet(test_path).show()

Spark alive? True
Write OK: True
+---+-----+
| id|  txt|
+---+-----+
|  1|hello|
|  2|world|
+---+-----+



In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# ================
# 1. 强化版 SparkSession
# ================
spark = (
    SparkSession.builder
        .appName("5400-news-bronze")
        .master("local[*]")
        .config("spark.driver.memory", "8g")         # 至少 8GB
        .config("spark.executor.memory", "8g")       # local 模式 = driver
        .config("spark.sql.shuffle.partitions", "200")   # 提高并行度
        .config("spark.default.parallelism", "200")
        .getOrCreate()
)

print("Spark OK:", spark.version)

# ================
# 2. 读取 CSV
# ================
csv_path = r"D:\Columbia\Fall2025\5400\project\All_external.csv"

df = (
    spark.read
        .format("csv")
        .options(
            header='true',
            inferSchema='true',
            treatEmptyValuesAsNulls='true'
        )
        .load(csv_path)
)

print("CSV rows:", df.count())

# ================
# 3. Bronze 分层路径
# ================
base_dir = r"D:\Columbia\Fall2025\5400\project\layer"
bronze_path = os.path.join(base_dir, "bronze")

os.makedirs(bronze_path, exist_ok=True)

# ================
# 4. repartition 200 → 避免 OOM
# ================
df_re = df.repartition(200)

# ================
# 5. 写 parquet（不会挂）
# ================
(
    df_re.write
        .mode("overwrite")
        .parquet(bronze_path)
)

print("✅ Bronze 层写入完成 →", bronze_path)


Spark OK: 3.3.1
CSV rows: 29984720
✅ Bronze 层写入完成 → D:\Columbia\Fall2025\5400\project\layer\bronze


In [2]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Article_title: string (nullable = true)
 |-- Stock_symbol: string (nullable = true)
 |-- Url: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- Article: string (nullable = true)
 |-- Lsa_summary: string (nullable = true)
 |-- Luhn_summary: string (nullable = true)
 |-- Textrank_summary: string (nullable = true)
 |-- Lexrank_summary: string (nullable = true)



#### Silver


In [ ]:
silver_path = os.path.join(base_layer_dir, "silver")

news_silver = (
    news_bronze
    # 统一日期格式：Date -> date（DateType）
    .withColumn("date", F.to_date(F.col("Date")))
    # 股票代码规范化：去空格 + 大写
    .withColumn("stock_symbol", F.upper(F.trim(F.col("Stock_symbol"))))
    # 正文清洗
    .withColumn("article_clean", F.trim(F.col("Article")))
    .filter(F.col("article_clean").isNotNull() & (F.col("article_clean") != ""))
    # 按 Url 去重（你也可以换成其他组合）
    .dropDuplicates(["Url"])
    .select(
        "date",
        "Article_title",
        "stock_symbol",
        "Url",
        "Publisher",
        "Author",
        "article_clean",
        "Lsa_summary",
        "Luhn_summary",
        "Textrank_summary",
        "Lexrank_summary"
    )
)

(
    news_silver.write
    .mode("overwrite")
    .parquet(silver_path)
)

print("✅ Silver 层写入完成：", silver_path)
news_silver.show(5, truncate=False)
